### Demo for langchain implementation

- thinking about an agent AI that integrates with the toolset
- see the reference documentation here: https://python.langchain.com/docs/tutorials/agents/
- in this case, using AWS Bedrock for DeepSeek model access using my named profile in `~/.aws/credentials` file

In [56]:
# Ensure your AWS credentials are configured
import langchain
from langchain.chat_models import init_chat_model
import pandas as pd
import getout_of_text_3 as got3

In [57]:
model = init_chat_model("us.deepseek.r1-v1:0", 
                        model_provider="bedrock_converse",
                        credentials_profile_name='atn-developer')

In [58]:
query = "Hi! What is the capital of France?"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'\n\nThe capital of France is **Paris**. Known for its rich history, cultural landmarks like the Eiffel Tower and the Louvre, and its role as the political and administrative heart of the country, Paris has been the capital since the Middle Ages. It remains the seat of the French government and a global hub for art, fashion, and diplomacy. 🇫🇷'

## Custom tool for langchain demo

- we can create a custom tool to query our corpus of SCOTUS cases from the LOC.gov database
- the model can then use this tool to answer questions about statutory interpretation cases


In [59]:
# read pdf scotus files
df = pd.read_json("loc_gov.json", lines=True)

df['key'] = df['filename'].apply(lambda x: x.split('usrep')[1][:3])
df['subkey'] = df['filename'].apply(lambda x: x.split('usrep')[1].split('.pdf')[0])

# Create a dictionary to hold the DataFrame contents
df_dict = {}

for _, row in df.iterrows():
    if row['key'] not in df_dict:
        df_dict[row['key']] = {}
    df_dict[row['key']][row['subkey']] = row['content']

# format scotus data for getout_of_text_3, similar to COCA keyword results
db_dict_formatted = {}
for volume, cases in df_dict.items():
    # Create a DataFrame for each volume with case text
    case_data = []
    for case_id, case_text in cases.items():
        case_data.append({'case_id': case_id, 'text': case_text})
    db_dict_formatted[volume] = pd.DataFrame(case_data)


In [60]:
loc_results = got3.search_keyword_corpus(
    keyword='ordinary meaning',
    db_dict=db_dict_formatted,
    case_sensitive=False,
    show_context=True,
    context_words=20,
    output="json"
)

In [61]:
# Drop keys with empty dicts and sort by keys (as integers)
filtered_sorted_results = {k: v for k, v in sorted(loc_results.items(), key=lambda item: int(item[0])) if v}
filtered_sorted_results

{'329': {'47': 'of the Court. 329 U. S. "arbitrary," as used by the District Court, was intended by it to have the **ordinary meaning** which that word has when used alone, we are unable to conclude on the record before us that the selection'},
 '332': {'27': 'revenue agents. Whatever the reason, the payment of more than is rightfully due is what characterizes an overpayment. That this **ordinary meaning** is the one intended by the authors of § 322 (b) (1) is quite evident from the legisla- tive history'},
 '335': {'31': 'clear its purpose, but when that purpose is made manifest in a manner that leaves no doubt according to the **ordinary meaning** of English speech, this Court, in disregarding it, is disregard- ing the limits of the judicial function which we all'},
 '337': {'10': 'regulation, we, in considering credits as property subject to vesting under the Trading with the Enemy Act, give it its **ordinary meaning** of the obligation due on accounting between parties to transacti

## AI Generated LangChain Tool Example

- this tool offers an AI implementation for summarizing questions about statutory interpretation cases
- it's of course a proof-of-concept, so please review the results accordingly!

```python

In [62]:
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional, Type, Dict, Any
import json
import asyncio

class ScotusAnalysisInput(BaseModel):
    """Input for SCOTUS case analysis tool."""
    filtered_results: str = Field(description="JSON string of filtered SCOTUS search results")
    keyword: str = Field(description="The keyword that was searched for in the cases")
    analysis_focus: Optional[str] = Field(
        default="general", 
        description="Focus of analysis: 'general', 'evolution', 'judicial_philosophy', or 'custom'"
    )

class ScotusAnalysisTool(BaseTool):
    """Tool for analyzing SCOTUS case search results using AI model."""
    
    name: str = "scotus_analysis"
    description: str = """
    Analyzes filtered SCOTUS case search results to identify interesting patterns, 
    trends, and insights about how legal concepts are used across different cases and time periods.
    Input should be JSON string of search results and the keyword searched.
    """
    args_schema: Type[BaseModel] = ScotusAnalysisInput
    model: Any = Field(exclude=True)  # Store the model instance
    
    def __init__(self, model, **kwargs):
        super().__init__(**kwargs)
        self.model = model
    
    def _run(self, filtered_results: str, keyword: str, analysis_focus: str = "general") -> str:
        """Synchronous version - handles event loop properly."""
        try:
            # Try to get the current event loop
            loop = asyncio.get_event_loop()
            if loop.is_running():
                # We're in a running event loop (like Jupyter), use create_task
                import nest_asyncio
                nest_asyncio.apply()
                return asyncio.run(self._arun(filtered_results, keyword, analysis_focus))
        except:
            pass
        
        # Fallback: use synchronous approach
        return self._sync_run(filtered_results, keyword, analysis_focus)
    
    def _sync_run(self, filtered_results: str, keyword: str, analysis_focus: str = "general") -> str:
        """Synchronous version for Jupyter compatibility."""
        try:
            # Parse the results
            results_data = json.loads(filtered_results) if isinstance(filtered_results, str) else filtered_results
            
            # Count cases and volumes
            total_cases = sum(len(cases) for cases in results_data.values())
            volumes = list(results_data.keys())
            
            # Create analysis prompt based on focus
            analysis_prompts = {
                "general": f"""
                Analyze these SCOTUS case search results for the keyword "{keyword}".
                
                Data summary:
                - Found in {len(volumes)} volumes: {', '.join(sorted(volumes, key=int))}
                - Total {total_cases} case occurrences
                
                Please provide insights on:
                1. How the usage of "{keyword}" has evolved over time
                2. Different contexts in which "{keyword}" appears
                3. Any notable patterns or trends
                4. Key judicial interpretations or applications
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """,
                
                "evolution": f"""
                Focus on the temporal evolution of "{keyword}" in SCOTUS cases.
                
                Analyze how the interpretation and usage of "{keyword}" has changed from volume {min(volumes, key=int)} to {max(volumes, key=int)}.
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """,
                
                "judicial_philosophy": f"""
                Analyze how different judicial philosophies approach "{keyword}".
                
                Look for patterns that might indicate originalist vs. living constitution approaches, or other judicial philosophies.
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """,
                
                "custom": f"""
                Provide a comprehensive analysis of "{keyword}" in these SCOTUS cases.
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """
            }
            
            prompt = analysis_prompts.get(analysis_focus, analysis_prompts["general"])
            
            # Get AI analysis using synchronous invoke
            response = self.model.invoke([{"role": "user", "content": prompt}])
            
            return response.content
            
        except Exception as e:
            return f"Error analyzing SCOTUS results: {str(e)}"
    
    async def _arun(self, filtered_results: str, keyword: str, analysis_focus: str = "general") -> str:
        """Analyze SCOTUS search results asynchronously."""
        
        try:
            # Parse the results
            results_data = json.loads(filtered_results) if isinstance(filtered_results, str) else filtered_results
            
            # Count cases and volumes
            total_cases = sum(len(cases) for cases in results_data.values())
            volumes = list(results_data.keys())
            
            # Create analysis prompt based on focus
            analysis_prompts = {
                "general": f"""
                Analyze these SCOTUS case search results for the keyword "{keyword}".
                
                Data summary:
                - Found in {len(volumes)} volumes: {', '.join(sorted(volumes, key=int))}
                - Total {total_cases} case occurrences
                
                Please provide insights on:
                1. How the usage of "{keyword}" has evolved over time
                2. Different contexts in which "{keyword}" appears
                3. Any notable patterns or trends
                4. Key judicial interpretations or applications
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """,
                
                "evolution": f"""
                Focus on the temporal evolution of "{keyword}" in SCOTUS cases.
                
                Analyze how the interpretation and usage of "{keyword}" has changed from volume {min(volumes, key=int)} to {max(volumes, key=int)}.
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """,
                
                "judicial_philosophy": f"""
                Analyze how different judicial philosophies approach "{keyword}".
                
                Look for patterns that might indicate originalist vs. living constitution approaches, or other judicial philosophies.
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """,
                
                "custom": f"""
                Provide a comprehensive analysis of "{keyword}" in these SCOTUS cases.
                
                Results: {json.dumps(results_data, indent=2)[:3000]}...
                """
            }
            
            prompt = analysis_prompts.get(analysis_focus, analysis_prompts["general"])
            
            # Get AI analysis
            response = await self.model.ainvoke([{"role": "user", "content": prompt}])
            
            return response.content
            
        except Exception as e:
            return f"Error analyzing SCOTUS results: {str(e)}"

# Create the tool instance
scotus_tool = ScotusAnalysisTool(model=model)

In [ ]:
# Test the tool with our filtered results
results_json = json.dumps(filtered_sorted_results)

# Run analysis
analysis = scotus_tool._run(
    filtered_results=results_json,
    keyword="ordinary meaning",
    analysis_focus="general"
)


[{'type': 'text', 'text': '\n\n### Analysis of SCOTUS Case References to "Ordinary Meaning"\n\n#### 1. **Evolution of Usage Over Time**  \nThe term "ordinary meaning" has been consistently invoked in SCOTUS decisions since at least the mid-20th century (e.g., Volume 329, likely from the 1940s) through recent volumes (e.g., Volume 570, circa 2020). Key trends include:  \n- **Early Use**: Initially, the term appeared sporadically, often as a secondary tool to resolve ambiguities (e.g., tax law in Volume 332 (1947) and immigration in Volume 347 (1951)).  \n- **Late 20th to 21st Century Surge**: The frequency increases notably in volumes from the 1990s onward (e.g., 500s and 500s), aligning with the rise of **textualism** as a dominant judicial philosophy. Justices like Scalia emphasized "ordinary meaning" to prioritize statutory text over legislative history, reflecting a broader shift toward judicial restraint and textual fidelity.  \n\n#### 2. **Contexts of Usage**  \nThe term appears a

In [72]:
# The analysis result is a string, not a list
print("Analysis type:", type(analysis))
print("Analysis content:", analysis[0]['text'])

Analysis type: <class 'list'>
Analysis content: 

### Analysis of SCOTUS Case References to "Ordinary Meaning"

#### 1. **Evolution of Usage Over Time**  
The term "ordinary meaning" has been consistently invoked in SCOTUS decisions since at least the mid-20th century (e.g., Volume 329, likely from the 1940s) through recent volumes (e.g., Volume 570, circa 2020). Key trends include:  
- **Early Use**: Initially, the term appeared sporadically, often as a secondary tool to resolve ambiguities (e.g., tax law in Volume 332 (1947) and immigration in Volume 347 (1951)).  
- **Late 20th to 21st Century Surge**: The frequency increases notably in volumes from the 1990s onward (e.g., 500s and 500s), aligning with the rise of **textualism** as a dominant judicial philosophy. Justices like Scalia emphasized "ordinary meaning" to prioritize statutory text over legislative history, reflecting a broader shift toward judicial restraint and textual fidelity.  

#### 2. **Contexts of Usage**  
The ter

## Test the SCOTUS Analysis Tool

Now let's test our custom tool with different analysis focuses:

In [65]:
# Try different analysis focuses

# 1. Evolution analysis
print("=== EVOLUTION ANALYSIS ===")
evolution_analysis = scotus_tool._run(
    filtered_results=results_json,
    keyword="ordinary meaning",
    analysis_focus="evolution"
)

=== EVOLUTION ANALYSIS ===


In [73]:
print(evolution_analysis[0]['text'])
print("\n" + "="*50 + "\n")



The interpretation and usage of "ordinary meaning" in SCOTUS cases from Volumes 329 to 355 (spanning the 1940s–1950s) reveal a nuanced evolution in judicial reasoning, balancing textualism with contextual pragmatism. Below is the analysis of the provided data and inferred trends:

---

### **Key Observations in Volumes 329–355**
1. **Foundational Reliance on Plain Language**  
   - Early cases (e.g., 329, 332, 335) treat "ordinary meaning" as the default starting point, rooted in common understanding. For example:
     - In **332 U.S. 27**, "overpayment" is defined by its "ordinary meaning" (paying more than owed), reinforced by legislative history.
     - **335 U.S. 31** emphasizes adherence to "ordinary meaning of English speech" unless statutory purpose is unclear.

2. **Contextual Overrides**  
   - By the 340s–350s, courts increasingly acknowledge that context (e.g., contractual terms, statutory purpose) may override ordinary meaning:
     - **339 U.S. 27** notes that contractua

In [67]:
# 2. Judicial philosophy analysis
print("=== JUDICIAL PHILOSOPHY ANALYSIS ===")
philosophy_analysis = scotus_tool._run(
    filtered_results=results_json,
    keyword="ordinary meaning",
    analysis_focus="judicial_philosophy"
)
print(philosophy_analysis)

=== JUDICIAL PHILOSOPHY ANALYSIS ===
[{'type': 'text', 'text': '\n\n**Analysis of Judicial Philosophies Through the Use of "Ordinary Meaning"**\n\nThe provided court citations predominantly reflect **textualist and originalist approaches**, with limited instances suggesting contextual flexibility. Here\'s a breakdown of the patterns:\n\n### **1. Originalist/Textualist Patterns**\n- **Focus on Plain Text**: \n  - Cases like *335 U.S.* and *338 U.S.* emphasize adhering to the "ordinary meaning of English speech" or statutory words, rejecting broader interpretations. This aligns with textualism, which prioritizes the text’s literal meaning.\n  - *340 U.S.* explicitly rejects a "broader definition" that distorts ordinary meaning, reflecting a commitment to textual fidelity.\n\n- **Legislative Intent and Historical Context**:\n  - *332 U.S.* uses legislative history to confirm the "ordinary meaning" of "overpayment," a method originalists employ to validate textual interpretations.\n  - *34

In [74]:
print(philosophy_analysis[0]['text'])
print("\n" + "="*50 + "\n")



**Analysis of Judicial Philosophies Through the Use of "Ordinary Meaning"**

The provided court citations predominantly reflect **textualist and originalist approaches**, with limited instances suggesting contextual flexibility. Here's a breakdown of the patterns:

### **1. Originalist/Textualist Patterns**
- **Focus on Plain Text**: 
  - Cases like *335 U.S.* and *338 U.S.* emphasize adhering to the "ordinary meaning of English speech" or statutory words, rejecting broader interpretations. This aligns with textualism, which prioritizes the text’s literal meaning.
  - *340 U.S.* explicitly rejects a "broader definition" that distorts ordinary meaning, reflecting a commitment to textual fidelity.

- **Legislative Intent and Historical Context**:
  - *332 U.S.* uses legislative history to confirm the "ordinary meaning" of "overpayment," a method originalists employ to validate textual interpretations.
  - *347 U.S.* notes Congress did not intend to deviate from the term "entry’s" ordin

## AWS Cost Explorer -- Monitoring Bedrock Costs

Useful `jq` filters for your AWS Cost Explorer output:

> notably costs won't show up immediately, so you may need to wait a day or two after incurring costs to see them reflected in Cost Explorer.

```bash
# Filter for Bedrock services specifically from all services with costs
named_profile=atn-developer
region=us-east-1
start_date=2025-08-30
end_date=2025-10-01

aws ce get-cost-and-usage --time-period Start=$start_date,End=$end_date --granularity DAILY --metrics "BlendedCost" --group-by Type=DIMENSION,Key=SERVICE --region $region --profile=$named_profile | jq '.ResultsByTime[].Groups[] | select(.Keys[0] | test("Bedrock"; "i")) | {service: .Keys[0], cost: .Metrics.BlendedCost.Amount}'
```

In [77]:
# Filter for Bedrock services specifically from all services with costs
!aws ce get-cost-and-usage --time-period Start=2025-09-30,End=2025-10-01 --granularity DAILY --metrics "BlendedCost" --group-by Type=DIMENSION,Key=SERVICE --region us-east-1 --profile=atn-developer | jq '.ResultsByTime[].Groups[] | select(.Keys[0] | test("Bedrock"; "i")) | {service: .Keys[0], cost: .Metrics.BlendedCost.Amount}'